In [0]:
# import the required modules
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, bias=False)#input:28x28x1, output:26x26x16, RF:3x3
        self.conv1_bn1 = nn.BatchNorm2d(16)
        self.conv1_drop1 = nn.Dropout2d(0.03)

        self.conv2 = nn.Conv2d(16, 16, 3, bias=False)#input:26x26x16, output:24x24x16, RF:5x5 
        self.conv2_bn2 = nn.BatchNorm2d(16)
        self.conv2_drop2 = nn.Dropout2d(0.03)
        self.conv3 = nn.Conv2d(16, 16, 3, bias=False)#input:24x24x16, output:22x22x16, RF:7x7
        self.conv3_bn3 = nn.BatchNorm2d(16)
        self.conv3_drop3 = nn.Dropout2d(0.03)

        self.pool1 = nn.MaxPool2d(2, 2)#input:22x22x16, output:11x11x16, RF:8x8  
        self.conv4 = nn.Conv2d(16, 16, 1, bias=False)#input:11x11x16, output:11x11x16, RF:8x8
        self.conv4_bn4 = nn.BatchNorm2d(16)
        self.conv4_drop4 = nn.Dropout2d(0.03)

        self.conv5 = nn.Conv2d(16, 24, 3, bias=False)#input:11x11x16, output:9x9x24, RF:12x12 
        self.conv5_bn5 = nn.BatchNorm2d(24)
        self.conv5_drop5 = nn.Dropout2d(0.03)
        self.conv6 = nn.Conv2d(24, 24, 3, bias=False)#input:9x9x24, output:7x7x24, RF:16x16
        self.conv6_bn6 = nn.BatchNorm2d(24)
        self.conv6_drop6 = nn.Dropout2d(0.03)
        
        self.conv7 = nn.Conv2d(24, 10, 1, bias=False)#input:7x7x24, output:7x7x10, RF:16x16
        self.gap1 = nn.AvgPool2d(kernel_size=7)#input:7x7x10, output:1x1x10 (or 1x10, 1-D data format now)
        
    def forward(self, x):
        x = F.relu(self.conv1_drop1(self.conv1_bn1(self.conv1(x))))#First Layer Separate
        x = F.relu(self.conv3_drop3(self.conv3_bn3(self.conv3(F.relu(self.conv2_drop2(self.conv2_bn2(self.conv2(x))))))))#First Block
        x = F.relu(self.conv4_drop4(self.conv4_bn4(self.conv4(self.pool1(x)))))#First transition block
        x = F.relu(self.conv6_drop6(self.conv6_bn6(self.conv6(F.relu(self.conv5_drop5(self.conv5_bn5(self.conv5(x))))))))#Second Block 
        x = self.conv7(x)#no ReLU at 2nd Transition Block O/p(just 1x1, without a maxpool towards the end here)leading to GAP & beyond) 
        x = self.gap1(x) # Global Average Pool layer
        x = x.view(-1, 10)#Num of classes = 10 (digits to be identified)
        return F.log_softmax(x, dim=1)

In [44]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
       BatchNorm2d-2           [-1, 16, 26, 26]              32
         Dropout2d-3           [-1, 16, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           2,304
       BatchNorm2d-5           [-1, 16, 24, 24]              32
         Dropout2d-6           [-1, 16, 24, 24]               0
            Conv2d-7           [-1, 16, 22, 22]           2,304
       BatchNorm2d-8           [-1, 16, 22, 22]              32
         Dropout2d-9           [-1, 16, 22, 22]               0
        MaxPool2d-10           [-1, 16, 11, 11]               0
           Conv2d-11           [-1, 16, 11, 11]             256
      BatchNorm2d-12           [-1, 16, 11, 11]              32
        Dropout2d-13           [-1, 16, 11, 11]               0
           Conv2d-14             [

In [0]:
torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [47]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0882, Accuracy: 9798/10000 (98%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0526, Accuracy: 9862/10000 (99%)



  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0192, Accuracy: 9939/10000 (99%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0202, Accuracy: 9934/10000 (99%)



loss=0.004588708281517029 batch_id=937: 100%|██████████| 938/938 [00:15<00:00, 62.29it/s]



Test set: Average loss: 0.0171, Accuracy: 9950/10000 (100%)

